In [1]:
from pprint import pprint
from pathlib import Path

import duckdb
import ipywidgets as widgets
import json

from mosaic_widget import MosaicWidget

In [2]:
# Optional: enable logging in Mosaic widget
import logging

logging.basicConfig(level=logging.INFO)

In [3]:
# Change working directory to mosaic root
%cd ../..

/Users/dominik/Developer/mosaic


/Users/dominik/Library/Application Support/hatch/env/virtual/mosaic-widget/flBMdGv2/mosaic-widget/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
con = duckdb.connect()
con.execute("LOAD httpfs");

In [5]:
mosaic = MosaicWidget(con=con)

In [6]:
def get_specs(folder_path):
    folder = Path(folder_path)
    for file_path in folder.glob("*.json"):
        data = json.loads(file_path.read_text())
        meta = data.get("meta")
        if meta:
            yield (meta.get("title"), file_path)

In [7]:
weather = Path("specs/json/weather.json")

dropdown = widgets.Dropdown(
    options=get_specs("specs/json"),
    value=weather,
    description="Example:",
)


def on_change(change):
    open_spec(change["new"])


def open_spec(spec):
    mosaic.spec = json.loads(spec.read_text())


dropdown.observe(on_change, "value")

open_spec(weather)

output = widgets.Output()


@output.capture(clear_output=True)
def handle_change(change):
    pprint(change.new)


mosaic.observe(handle_change, names=["params"])

In [8]:
widgets.VBox([dropdown, mosaic, output])